In [300]:
import numpy as np
import pandas as pd 
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin

In [301]:
df = pd.read_csv("./playground-series-s4e9/train.csv",index_col='id')
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
id,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [302]:
df.isna().sum()

brand               0
model               0
model_year          0
milage              0
fuel_type        5083
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
price               0
dtype: int64

In [303]:
df["fuel_type"].value_counts()

fuel_type
Gasoline          165940
Hybrid              6832
E85 Flex Fuel       5406
Diesel              3955
–                    781
Plug-In Hybrid       521
not supported         15
Name: count, dtype: int64

# Custom transformations automisation

The following class will preform automizations to the data.
It will be compatible with the scikit learn Pipeline and Transformer objects, making sure the entire pipeline is done automatically on the test data as well.

## Handling Null values in the fuel_type and accident attributes

Since the number of NUlls is very low (3%, 1.5% of the data respectively) we have 2 options:

Option 1 - drop those entries and train the model without them.

Option 2 - Replacing them with another value:
1. The most frequent value in the dataset.
2. A value indicating "unknown".

## Handling Null values in the clean_title attribute

Since the number of NUlls is more significant, we have 2 different options:

Option 1 - drop those entries, since the column would only have one value remaining, it would not contribute to the model training - we would have to drop the attribute entirely.

Option 2 - Replacing them with another value - "No".

We will choose option 2.

In [ ]:
def ImputerOne(X):
	X = X.copy()
	
	# Drop null values based on specific conditions
	X = X[X['fuel_type'].notna() & (X['fuel_type'] != 'not supported') & (X['fuel_type'] != '–')]
	X = X[X['accident'].notna()]
	
	# Fill missing values in 'clean_title' column
	X['clean_title'] = X['clean_title'].fillna("No")
	
	return X

In [ ]:
def ImputerTwo(X):
	X = X.copy()
	
	X["fuel_type"] = X["fuel_type"].replace('not supported', "Gasoline")
	X["fuel_type"] = X["fuel_type"].replace('–', "Gasoline")
	X["fuel_type"] = X["fuel_type"].replace(pd.NA, "Gasoline")

	X['accident'] = X['accident'].fillna("Unknown")

	X['clean_title'] = X['clean_title'].fillna("No")
	
	return X

In [339]:
def EngineTransformer(X):
	X = X.copy()

	# In case the engine column did not contain the string
	# X['horse_power'] = -1
	# X['tank_size'] = -1
	# X['num_cylinders'] = -1

	X['horse_power'] = X['engine'].str.extract(r'(\d+\.?\d*)HP').astype(float)
	X['tank_size'] = X['engine'].str.extract(r'(\d+\.?\d*)L').astype(float)
	X['num_cylinders'] = X['engine'].str.extract(r'(\d+)\sCylinder').astype(float) 
	   
	# Drop the original 'engine' column
	X.drop('engine', axis=1, inplace=True)
	return X

In [340]:
def TransmissionTransformer(X):
	# Initialize all attributes to False
    X = X.copy()

    X['automatic'] = False
    X['manual'] = False
    X['dual'] = False
    
    # Categorize as 'dual' if it contains 'Dual'
    X.loc[X['transmission'].str.contains(r'\b(Dual|At/Mt)\b', case=False, na=False), 'dual'] = True
    
    # Ensure 'dual' is boolean before using ~ operator
    X['dual'] = X['dual'].astype(bool)
    # Categorize as 'automatic' if it contains AT, A/T, CVT, or Automatic (but not Dual)
    X.loc[(X['transmission'].str.contains(r'\b(AT|A/T|CVT|Automatic)\b', case=False, na=False)) & ~X['dual'], 'automatic'] = True
    # Categorize as 'manual' if it contains MT, M/T, or Manual (but not Dual)
    X.loc[(X['transmission'].str.contains(r'\b(MT|M/T|Manual)\b', case=False, na=False)) & ~X['dual'], 'manual'] = True

    # Drop the original 'transmission' column
    X.drop('transmission', axis=1, inplace=True)
    
    return X

In [341]:
def preprocessor(X, option):
	if option == 1:
		X = ImputerOne(X)
	if option == 2:
		# X = ImputerTwo(X)
		pass
	X = EngineTransformer(X)
	X = TransmissionTransformer (X)

	# ColorTransformer, etc...

	# The test set does not have the target column
	if 'price' in ((df.columns).to_list()):
		# reorder the df to have price in the rightmost column
		# X = OrderDataframe
		pass
	
	return(X)

In [342]:
df_proc1 = preprocessor(df,1)
df_proc2 = preprocessor(df,2)

/tmp/ipykernel_7358/771887330.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  X.loc[X['transmission'].str.contains(r'\b(Dual|At/Mt)\b', case=False, na=False), 'dual'] = True
/tmp/ipykernel_7358/771887330.py:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  X.loc[(X['transmission'].str.contains(r'\b(AT|A/T|CVT|Automatic)\b', case=False, na=False)) & ~X['dual'], 'automatic'] = True
/tmp/ipykernel_7358/771887330.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  X.loc[(X['transmission'].str.contains(r'\b(MT|M/T|Manual)\b', case=False, na=False)) & ~X['dual'], 'manual'] = True
/tmp/ipykernel_7358/771887330.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the g

In [343]:
print (f'The length of df_proc1 is: {len(df_proc1)}')
print (f'The length of df_proc2 is: {len(df_proc2)}')

The length of df_proc1 is: 180288
The length of df_proc2 is: 188533


In [344]:
df_proc1.head()

,brand,model,model_year,milage,fuel_type,ext_col,int_col,accident,clean_title,price,horse_power,tank_size,num_cylinders,automatic,manual,dual
id,,,,,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,Yellow,Gray,None reported,Yes,4200,172.0,1.6,4.0,True,False,False
1,Lincoln,LS V8,2002,143250,Gasoline,Silver,Beige,At least 1 accident or damage reported,Yes,4999,252.0,3.9,8.0,True,False,False
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,Blue,Gray,None reported,Yes,13900,320.0,5.3,8.0,True,False,False
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,Black,Black,None reported,Yes,45000,420.0,5.0,8.0,False,False,True
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,Black,Beige,None reported,Yes,97500,208.0,2.0,4.0,True,False,False


In [345]:
df_proc1.isna().sum()

brand                0
model                0
model_year           0
milage               0
fuel_type            0
ext_col              0
int_col              0
accident             0
clean_title          0
price                0
horse_power      30826
tank_size         8433
num_cylinders    31250
automatic            0
manual               0
dual                 0
dtype: int64

In [348]:
df_proc1[df_proc1['horse_power'].isna()].head(1)

,brand,model,model_year,milage,fuel_type,ext_col,int_col,accident,clean_title,price,horse_power,tank_size,num_cylinders,automatic,manual,dual
id,,,,,,,,,,,,,,,,
8,Ford,F-150 XLT,2020,38352,Gasoline,Snowflake White Pearl Metallic,Black,None reported,Yes,62890,NaN,2.7,NaN,True,False,False


In [328]:
df.head(9).tail(1)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
id,,,,,,,,,,,,
8,Ford,F-150 XLT,2020,38352,Gasoline,2.7L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,Snowflake White Pearl Metallic,Black,None reported,Yes,62890


In [313]:
df_proc2.isna().sum()

brand                0
model                0
model_year           0
milage               0
fuel_type         5083
ext_col              0
int_col              0
accident          2452
clean_title      21419
price                0
horse_power      33259
tank_size        14214
num_cylinders    37855
automatic            0
manual               0
dual                 0
dtype: int64